In [11]:
import base64
import requests
import os
import json
import openai
import re
from tqdm import tqdm

# Define base URL and model configuration for OpenAI API
base_url = ""
api_key = ""
model = "sora_image"  # Use your actual model name here, e.g., "gpt-4o-image-vip"
client = openai.OpenAI(
    base_url=base_url,
    api_key=api_key,
)


def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')


def download_image(image_url, save_path):
    try:
        # Clean the URL and extract the valid part
        image_url = image_url.strip().rstrip(')')
        # Use regex to extract the full URL (starting with http/https and including image extensions)
        url_match = re.search(r'(https?://[^\s]+\.(?:png|jpg|jpeg|gif|bmp|webp))', image_url, re.IGNORECASE)
        if not url_match:
            print(f"Invalid URL format: {image_url}")
            return
        image_url = url_match.group(1)

        headers = {
            "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.0.0 Safari/537.36"
        }
        response = requests.get(image_url, headers=headers, timeout=15)
        if response.status_code == 200:
            with open(save_path, 'wb') as img_file:
                img_file.write(response.content)
            print(f"Image saved to {save_path}")  # Corrected here
        else:
            print(f"Download failed (Status {response.status_code}): {image_url}")
    except Exception as e:
        print(f"Download error: {e}")



def generate_and_save_image(image_paths, folder_name, output_json_path, prompt, model, client, output_image_folder, pair_index):
    base64_images = [encode_image(image_path) for image_path in image_paths]

    # Creating a unique filename based on the image pair index (e.g., result_0_1, result_2_3)
    image_filename = f"result_{pair_index * 2}_{pair_index * 2 + 1}_generated.jpg"
    save_image_path = os.path.join(output_image_folder, folder_name, image_filename)
    
    os.makedirs(os.path.dirname(save_image_path), exist_ok=True)

    configure = {
        "model": model,
        "messages": [
            {"role": "system", "content": "You are a helpful assistant."},
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": prompt},
                    {"type": "image_url", "image_url": {"url": f"data:image/jpeg;base64,{base64_images[0]}"}}  # Image 1
                ]
            }
        ]
    }

    # Generating the image based on the provided prompt
    #response = client.chat.completions.create(**configure)
    response = client.chat.completions.create(
    **configure,
)
    data = response.choices[0].message.content

    # Downloading the generated image
    download_image(data, save_image_path)

    output_log = {"image_paths": image_paths, "response": data}
    with open(output_json_path, 'a') as json_file:
        json.dump(output_log, json_file, indent=4)
        json_file.write("\n")


def process_images_from_folder(folder_path, output_json_path, prompt_groups, model, client, output_image_folder):
    folder_name = os.path.basename(folder_path)  # Extract the folder name (e.g., "100831")
    image_files = [f for f in os.listdir(folder_path) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]

    # Sort the images based on the numeric part of the filename
    image_files.sort(key=lambda x: int(re.search(r'part_(\d+)', x).group(1)))  # Sort by the number after 'part_'

    # Process images in pairs of (0,1), (2,3), (4,5), ...
    for i in tqdm(range(0, len(image_files), 2), desc=f"Processing Images for Folder {folder_name}", unit="pair"):
        # Get the two images (pair) corresponding to a particular subject
        image_paths = [os.path.join(folder_path, image_files[i]), os.path.join(folder_path, image_files[i + 1])]
        print(f"Processing pair: {image_files[i]} and {image_files[i + 1]} for folder {folder_name}")
        
        # Determine the correct prompt for the pair (0-1 for upper body, 2-3 for lower body, etc.)
        pair_index = i // 2
        prompt = prompt_groups[pair_index]  # Assuming prompt_groups matches the order 0-1, 2-3, 4-5
        generate_and_save_image(image_paths, folder_name, output_json_path, prompt, model, client, output_image_folder, pair_index)
        print(f"Processed pair: {image_paths[0]} and {image_paths[1]} for folder {folder_name}")


# Example Configuration (you need to replace these with actual parameters)
folder_path = "/data_lg/keru/DP_gpt_250502/result/100832"
output_json_path = "/data_lg/keru/DP_gpt_250502/output_log.json"
output_image_folder = "/data_lg/keru/DP_gpt_250502/generated_images"
os.makedirs(output_image_folder, exist_ok=True)

# Define prompt groups for each image pair (0-1, 2-3, 4-5)
prompt_groups = [
    "这是一件上装的前后，请参照为我重新生成上装的前后面,并将结果拼接呈现 ,高清,还原花纹、颜色、样式,背景为白色,细节清晰,图片规格,高1024,宽768*2（上装前面宽为768，上装后面为768），生成结果尽量保持居中，衣服长宽比需合理。",
    "这是一件下装的前后，请参照为我重新生成下装的前后面 并将结果拼接呈现 ,高清,还原花纹、颜色、样式,背景为白色,细节清晰,图片规格,高1024,宽768*2（下装前面宽为768，下装后面为768），生成结果尽量保持居中，衣服长宽比需合理。",
    "这是一件套装的前后，套装可能是裙子，背带裤，请将他看作一件整体的衣服。请参照输入为我重新生成套装的前后面 并将结果拼接呈现 ,高清,还原花纹、颜色、样式,背景为白色,细节清晰,图片规格,高1024,宽768*2(套装前面宽为768，套装后面为768），生成结果尽量保持居中，衣服长宽比需合理。"
]

# Process images from the folder
process_images_from_folder(folder_path, output_json_path, prompt_groups, model, client, output_image_folder)



Processing Images for Folder 100832:   0%|          | 0/1 [00:00<?, ?pair/s]

Processing pair: part_4.png and part_5.png for folder 100832


Processing Images for Folder 100832: 100%|██████████| 1/1 [01:35<00:00, 95.16s/pair]

Download error: name 'save_image_path' is not defined
Processed pair: /data_lg/keru/DP_gpt_250502/result/100832/part_4.png and /data_lg/keru/DP_gpt_250502/result/100832/part_5.png for folder 100832


In [13]:
import base64
import requests
import os
import json
import openai
import re
from tqdm import tqdm

# Define base URL and model configuration for OpenAI API
base_url = ""
api_key = ""
model = "sora_image"  # Use your actual model name here, e.g., "gpt-4o-image-vip"
client = openai.OpenAI(
    base_url=base_url,
    api_key=api_key,
)


def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')


def download_image(image_url, save_path):
    try:
        # Clean the URL and extract the valid part
        image_url = image_url.strip().rstrip(')')
        # Use regex to extract the full URL (starting with http/https and including image extensions)
        url_match = re.search(r'(https?://[^\s]+\.(?:png|jpg|jpeg|gif|bmp|webp))', image_url, re.IGNORECASE)
        if not url_match:
            print(f"Invalid URL format: {image_url}")
            return
        image_url = url_match.group(1)

        headers = {
            "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.0.0 Safari/537.36"
        }
        response = requests.get(image_url, headers=headers, timeout=15)
        if response.status_code == 200:
            with open(save_path, 'wb') as img_file:
                img_file.write(response.content)
            print(f"Image saved to {save_path}")  # Corrected here
        else:
            print(f"Download failed (Status {response.status_code}): {image_url}")
    except Exception as e:
        print(f"Download error: {e}")


def generate_and_save_image(image_paths, folder_name, output_json_path, prompt, model, client, output_image_folder, pair_index, image_files):
    base64_images = [encode_image(image_path) for image_path in image_paths]

    # Extract numbers from the image files (e.g., part_4.png -> 4, part_5.png -> 5)
    numbers = [re.search(r'part_(\d+)', image_file).group(1) for image_file in image_files]

    # Create a unique filename based on the numbers extracted from the input file names (e.g., result_4_5_generated.jpg)
    image_filename = f"result_{numbers[0]}_{numbers[1]}_generated.jpg"
    save_image_path = os.path.join(output_image_folder, folder_name, image_filename)
    
    os.makedirs(os.path.dirname(save_image_path), exist_ok=True)

    configure = {
        "model": model,
        "messages": [
            {"role": "system", "content": "You are a helpful assistant."},
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": prompt},
                    {"type": "image_url", "image_url": {"url": f"data:image/jpeg;base64,{base64_images[0]}"}}  # Image 1
                ]
            }
        ]
    }

    # Generate the image using the model (this part remains unchanged)
    response = client.chat.completions.create(**configure)
    data = response.choices[0].message.content

    # Downloading the generated image
    download_image(data, save_image_path)

    output_log = {"image_paths": image_paths, "response": data}
    with open(output_json_path, 'a') as json_file:
        json.dump(output_log, json_file, indent=4)
        json_file.write("\n")


def process_images_from_folder(folder_path, output_json_path, prompt_groups, model, client, output_image_folder):
    folder_name = os.path.basename(folder_path)  # Extract the folder name (e.g., "100832")
    image_files = [f for f in os.listdir(folder_path) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]

    # Sort the images based on the numeric part of the filename
    image_files.sort(key=lambda x: int(re.search(r'part_(\d+)', x).group(1)))  # Sort by the number after 'part_'

    # Process images in pairs of (0,1), (2,3), (4,5), ...
    for i in tqdm(range(0, len(image_files), 2), desc=f"Processing Images for Folder {folder_name}", unit="pair"):
        # Get the two images (pair) corresponding to a particular subject
        image_paths = [os.path.join(folder_path, image_files[i]), os.path.join(folder_path, image_files[i + 1])]
        print(f"Processing pair: {image_files[i]} and {image_files[i + 1]} for folder {folder_name}")
        
        # Determine the correct prompt for the pair (0-1 for upper body, 2-3 for lower body, etc.)
        pair_index = i // 2
        prompt = prompt_groups[pair_index]  # Assuming prompt_groups matches the order 0-1, 2-3, 4-5
        generate_and_save_image(image_paths, folder_name, output_json_path, prompt, model, client, output_image_folder, pair_index, image_files[i:i+2])
        print(f"Processed pair: {image_paths[0]} and {image_paths[1]} for folder {folder_name}")


# Example Configuration (you need to replace these with actual parameters)
folder_path = "/data_lg/keru/DP_gpt_250502/result/100832"
output_json_path = "/data_lg/keru/DP_gpt_250502/output_log.json"
output_image_folder = "/data_lg/keru/DP_gpt_250502/generated_images"
os.makedirs(output_image_folder, exist_ok=True)

# Define prompt groups for each image pair (0-1, 2-3, 4-5)
prompt_groups = [
    "这是一件上装的前后，请参照为我重新生成上装的前后面,并将结果拼接呈现 ,高清,还原花纹、颜色、样式,背景为白色,细节清晰,图片规格,高1024,宽768*2（上装前面宽为768，上装后面为768），生成结果尽量保持居中，衣服长宽比需合理。",
    "这是一件下装的前后，请参照为我重新生成下装的前后面 并将结果拼接呈现 ,高清,还原花纹、颜色、样式,背景为白色,细节清晰,图片规格,高1024,宽768*2（下装前面宽为768，下装后面为768），生成结果尽量保持居中，衣服长宽比需合理。",
    "这是一件套装的前后，套装可能是裙子，背带裤，请将他看作一件整体的衣服。请参照输入为我重新生成套装的前后面 并将结果拼接呈现 ,高清,还原花纹、颜色、样式,背景为白色,细节清晰,图片规格,高1024,宽768*2(套装前面宽为768，套装后面为768），生成结果尽量保持居中，衣服长宽比需合理。"
]

# Process images from the folder
process_images_from_folder(folder_path, output_json_path, prompt_groups, model, client, output_image_folder)



Processing Images for Folder 100832:   0%|          | 0/1 [00:00<?, ?pair/s]

Processing pair: part_4.png and part_5.png for folder 100832


Processing Images for Folder 100832: 100%|██████████| 1/1 [01:29<00:00, 89.85s/pair]

Image saved to /data_lg/keru/DP_gpt_250502/generated_images/100832/result_4_5_generated.jpg
Processed pair: /data_lg/keru/DP_gpt_250502/result/100832/part_4.png and /data_lg/keru/DP_gpt_250502/result/100832/part_5.png for folder 100832


# 批量处理多个subject id

In [1]:
import base64
import requests
import os
import json
import openai
import re
from tqdm import tqdm

# Define base URL and model configuration for OpenAI API
base_url = ""
api_key = ""
model = "sora_image"  # Use your actual model name here, e.g., "gpt-4o-image-vip"
client = openai.OpenAI(
    base_url=base_url,
    api_key=api_key,
)


def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')


def download_image(image_url, save_path):
    try:
        # Clean the URL and extract the valid part
        image_url = image_url.strip().rstrip(')')
        # Use regex to extract the full URL (starting with http/https and including image extensions)
        url_match = re.search(r'(https?://[^\s]+\.(?:png|jpg|jpeg|gif|bmp|webp))', image_url, re.IGNORECASE)
        if not url_match:
            print(f"Invalid URL format: {image_url}")
            return
        image_url = url_match.group(1)

        headers = {
            "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.0.0 Safari/537.36"
        }
        response = requests.get(image_url, headers=headers, timeout=15)
        if response.status_code == 200:
            with open(save_path, 'wb') as img_file:
                img_file.write(response.content)
            print(f"Image saved to {save_path}")  # Corrected here
        else:
            print(f"Download failed (Status {response.status_code}): {image_url}")
    except Exception as e:
        print(f"Download error: {e}")


def generate_and_save_image(image_paths, folder_name, output_json_path, prompt, model, client, output_image_folder, pair_index, image_files):
    base64_images = [encode_image(image_path) for image_path in image_paths]

    # Extract numbers from the image files (e.g., part_4.png -> 4, part_5.png -> 5)
    numbers = [re.search(r'part_(\d+)', image_file).group(1) for image_file in image_files]

    # Create a unique filename based on the numbers extracted from the input file names (e.g., result_4_5_generated.jpg)
    image_filename = f"result_{numbers[0]}_{numbers[1]}_generated.jpg"
    save_image_path = os.path.join(output_image_folder, folder_name, image_filename)
    
    os.makedirs(os.path.dirname(save_image_path), exist_ok=True)

    configure = {
        "model": model,
        "messages": [
            {"role": "system", "content": "You are a helpful assistant."},
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": prompt},
                    {"type": "image_url", "image_url": {"url": f"data:image/jpeg;base64,{base64_images[0]}"}}  # Image 1
                ]
            }
        ]
    }

    # Generate the image using the model (this part remains unchanged)
    response = client.chat.completions.create(**configure)
    data = response.choices[0].message.content

    # Downloading the generated image
    download_image(data, save_image_path)

    output_log = {"image_paths": image_paths, "response": data}
    with open(output_json_path, 'a') as json_file:
        json.dump(output_log, json_file, indent=4)
        json_file.write("\n")


def process_images_from_folder(folder_path, output_json_path, prompt_groups, model, client, output_image_folder):
    # Iterate over all subject ID folders in the main folder
    subject_folders = [f for f in os.listdir(folder_path) if os.path.isdir(os.path.join(folder_path, f))]

    for folder_name in subject_folders:
        subject_folder_path = os.path.join(folder_path, folder_name)
        print(f"Processing images in folder {folder_name}")

        image_files = [f for f in os.listdir(subject_folder_path) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]

        # Sort the images based on the numeric part of the filename
        image_files.sort(key=lambda x: int(re.search(r'part_(\d+)', x).group(1)))  # Sort by the number after 'part_'

        # Process images in pairs of (0,1), (2,3), (4,5), ...
        for i in tqdm(range(0, len(image_files), 2), desc=f"Processing Images for Folder {folder_name}", unit="pair"):
            # Get the two images (pair) corresponding to a particular subject
            image_paths = [os.path.join(subject_folder_path, image_files[i]), os.path.join(subject_folder_path, image_files[i + 1])]
            print(f"Processing pair: {image_files[i]} and {image_files[i + 1]} for folder {folder_name}")
            
            # Determine the correct prompt for the pair (0-1 for upper body, 2-3 for lower body, etc.)
            pair_index = i // 2
            prompt = prompt_groups[pair_index]  # Assuming prompt_groups matches the order 0-1, 2-3, 4-5
            generate_and_save_image(image_paths, folder_name, output_json_path, prompt, model, client, output_image_folder, pair_index, image_files[i:i+2])
            print(f"Processed pair: {image_paths[0]} and {image_paths[1]} for folder {folder_name}")


# Example Configuration (you need to replace these with actual parameters)
folder_path = "/data_lg/keru/DP_gpt_250502/valid_result"
output_json_path = "/data_lg/keru/DP_gpt_250502/output_log.json"
output_image_folder = "/data_lg/keru/DP_gpt_250502/generated_images"
os.makedirs(output_image_folder, exist_ok=True)

# Define prompt groups for each image pair (0-1, 2-3, 4-5)
prompt_groups = [
    "这是一件上装的前后，请参照为我重新生成上装的前后面,并将结果拼接呈现 ,高清,还原花纹、颜色、样式,背景为白色,细节清晰,图片规格,高1024,宽768*2（上装前面宽为768，上装后面为768），生成结果尽量保持居中，衣服长宽比需合理。",
    "这是一件下装的前后，请参照为我重新生成下装的前后面 并将结果拼接呈现 ,高清,还原花纹、颜色、样式,背景为白色,细节清晰,图片规格,高1024,宽768*2（下装前面宽为768，下装后面为768），生成结果尽量保持居中，衣服长宽比需合理。",
    "这是一件套装的前后，套装可能是裙子，背带裤，请将他看作一件整体的衣服。请参照输入为我重新生成套装的前后面 并将结果拼接呈现 ,高清,还原花纹、颜色、样式,背景为白色,细节清晰,图片规格,高1024,宽768*2(套装前面宽为768，套装后面为768），生成结果尽量保持居中，衣服长宽比需合理。"
]

# Process images from the folder
process_images_from_folder(folder_path, output_json_path, prompt_groups, model, client, output_image_folder)


Processing images in folder 204089


Processing Images for Folder 204089:   0%|          | 0/1 [00:00<?, ?pair/s]

Processing pair: part_4.png and part_5.png for folder 204089


Processing Images for Folder 204089: 100%|██████████| 1/1 [01:18<00:00, 78.15s/pair]


Image saved to /data_lg/keru/DP_gpt_250502/generated_images/204089/result_4_5_generated.jpg
Processed pair: /data_lg/keru/DP_gpt_250502/valid_result/204089/part_4.png and /data_lg/keru/DP_gpt_250502/valid_result/204089/part_5.png for folder 204089
Processing images in folder 103214


...

Image saved to /data_lg/keru/DP_gpt_250502/generated_images/200206/result_2_3_generated.jpg
Processed pair: /data_lg/keru/DP_gpt_250502/valid_result/200206/part_2.png and /data_lg/keru/DP_gpt_250502/valid_result/200206/part_3.png for folder 200206
Processing images in folder 202844


Processing Images for Folder 202844:   0%|          | 0/2 [00:00<?, ?pair/s]

Processing pair: part_0.png and part_1.png for folder 202844


Processing Images for Folder 202844:  50%|█████     | 1/2 [01:25<01:25, 85.73s/pair]

Image saved to /data_lg/keru/DP_gpt_250502/generated_images/202844/result_0_1_generated.jpg
Processed pair: /data_lg/keru/DP_gpt_250502/valid_result/202844/part_0.png and /data_lg/keru/DP_gpt_250502/valid_result/202844/part_1.png for folder 202844
Processing pair: part_2.png and part_3.png for folder 202844
